# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)


In [ ]:
# assign the uk_food database to a variable name
db = mongo.uk_food

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments_list = db['establishments']
print(establishments_list)

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20

query = {'scores.Hygiene': 20}

results = establishments_list.find(query)
# use to print the results if needed:
# for result in results:
#     print(result)

# Use count_documents to display the number of documents in the result
Establishments_hygiene_20 = establishments_list.count_documents(query)

print("Number of establishments with hygiene score of 20: ", Establishments_hygiene_20)
# Display the first document in the results using pprint
results[0]


In [ ]:
# Convert the result to a Pandas DataFrame
Establishments_hygiene_20_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("No of rows:", len(Establishments_hygiene_20_df))
# Display the first 10 rows of the DataFrame
Establishments_hygiene_20_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': '4'}}

results = establishments_list.find(query)
# use to print the results if needed:
#for result in results:
#    print(result)

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint
pprint(establishments_list.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
london_gte4_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("No of rows:", len(london_gte4_df))

# Display the first 10 rows of the DataFrame
london_gte4_df.head(10)



### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'RatingValue': {'$regex':'5'}, 'geocode.latitude':{'$gte': (latitude - degree_search), '$lte': (latitude + degree_search)}, 'geocode.longitude':{'$gte': (longitude - degree_search), '$lte': (longitude + degree_search)} }
sort =  [('scores.Hygiene', 1)]

# Print the results
#results = establishments_list.find(query).limit(5)
results = establishments_list.find(query).sort(sort).limit(5)

for result in results:
    pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
pd.DataFrame(establishments_list.find(query).sort(sort).limit(5))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [154]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0

hygiene_match_query = {'$match': {'scores.Hygiene': 0}}

# 2. Groups the matches by Local Authority

group_authority_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}

# 3. Sorts the matches from highest to lowest
sort_descending = {'$sort': {'count': -1}}

# Print the number of documents in the result
pipeline = [hygiene_match_query, group_authority_query, sort_descending]

results = list(establishments_list.aggregate(pipeline))

# Print the first 10 results

pprint(results[0:10])

[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [157]:
# Convert the result to a Pandas DataFrame
pipeline_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
len(pipeline_df)
# Display the first 10 rows of the DataFrame
pipeline_df.head(10)

,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
